In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/typeform_responses.csv")

In [5]:
df['answers']

0    [{'field': {'id': 'AyomwPStZt76', 'type': 'sho...
1    [{'field': {'id': 'AyomwPStZt76', 'type': 'sho...
2    [{'field': {'id': 'AyomwPStZt76', 'type': 'sho...
3    [{'field': {'id': 'AyomwPStZt76', 'type': 'sho...
4    [{'field': {'id': 'AyomwPStZt76', 'type': 'sho...
5    [{'field': {'id': 'AyomwPStZt76', 'type': 'sho...
Name: answers, dtype: object